#  ThreeSameLetters

The solution given by [2018 TCO Algorithm Round 1B Editorials](https://www.topcoder.com/blog/2018-tco-algorithm-round-1b-editorials/) is as follow

In [1]:
def countStrings(L,S):
    t = [1, S-1]                
    for i in range(L):
        t.append((t[-1] + t[-2]) * (S-1))
    return S * sum(t[i]*t[L-3-i] for i in range(L-3+1)) % 1000000007

Very concise, however, notice that the space complexity is $O(L)$, there is still room to optimise by pre-computing t[n] and reverse it by t[n-2]=inv(S)*t[n-1]-t[n], so its not necessary to store all of them.

In [2]:
def xgcd(b, a):
    """
    Extended Euclidean algorithm.
    """
    x0, x1, y0, y1 = 1, 0, 0, 1
    while a != 0:
        q, b, a = b // a, a, b % a
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
    return  b, x0, y0

def count_strings(L,S):
    tt2 = t2=1
    tt1 = t1=S-1
    sum = 0
    M = 1000000007
    for i in range(L-2):
        t0 = (t1+t2)*(S-1)
        t1,t2 = t0,t1
    _,invS1,_ = xgcd((S-1),M)
    for i in range(L-2):
        t3 = invS1*t1 - t2
        tt0 = (tt1+tt2)*(S-1)       
        sum+=(t3*tt2) % M
        t2,t1 = t3,t2
        tt1,tt2 = tt0,tt1
    return sum * S % M

So far the time/space complexity are down to $O(L)$/$O(1)$.

Following content is to push the algorithm to extreme by cutting down more then 50% time complexity and also for myself to review old and explore new knowledges.

# Check List

[Diagonalizable matrix](https://en.wikipedia.org/wiki/Diagonalizable_matrix)

[Z-transform](https://en.wikipedia.org/wiki/Z-transform)

[Modular multiplicative inverse](https://en.wikipedia.org/wiki/Modular_multiplicative_inverse) 

[Extended Euclidean algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm)

# Zero block string

An one-block string can be represented as the combination of two zero-block starts with 2 same letters
Let $a$ as the number of different strings start with 2 same letters, $b$ as the number of different strings start with 2 different letters, $S+1$ represent the number of characters.
Then the recursion would be:

$$
\left\{
\begin{aligned}
a_n & =b_{n-1} \\
b_n & =S(a_{n-1}+b_{n-1}) \\
a_1 & = 0 \\
b_1 & = S + 1
\end{aligned}
\right.
$$

In [3]:
def enum_zero_block_strings(N,S):
    if N is 0:
        return []
    if N is 1:
        return range(0,S)
    if N is 2:
        return [ [i,j] for i in range(0,S+1) for j in range(0,S+1)]
    l = enum_zero_block_strings(N-1,S)
    return [
        [i] + x for x in l for i in range(0,S+1) if not (i==x[0] and x[0]==x[1])
    ]

def count_zero_block_strings(N,S):
    state = (0,S+1)
    while(N-1):
        next_state = (
            state[1],
            S*(state[0]+state[1])
        )
        state = next_state
        N-=1
    return state


N, S = 5, 5
sub_strings = enum_zero_block_strings(N,S)
[
    (
        len([x for x in sub_strings if len(x)>1 and x[0]==x[1]]),
        len([x for x in sub_strings if len(x)<=1 or x[0]!=x[1]]),
    ),
    count_zero_block_strings(N,S),
]


[(1050, 6150), (1050, 6150)]

The number sequence can be represented as the following linear discrete system:
$$
\begin{bmatrix}
a_n \\
b_n
\end{bmatrix}
=
\begin{bmatrix}
0 & 1 \\
S & S
\end{bmatrix}
\begin{bmatrix}
a_{n-1} \\
b_{n-1}
\end{bmatrix}
$$

Diagonalize the transform matrix using sympy library to reduce manual works


In [4]:
import sympy as sp
sp.init_printing(use_latex='mathjax')
sS = sp.Symbol('S')
A = sp.Matrix([
    [0,1],
    [sS,sS]
])
P,D = A.diagonalize()
P,D
#sp.N(D.subs(sS,S))

⎛                                        ⎡      ___________                   
⎜⎡        2                  2        ⎤  ⎢S   ╲╱ S⋅(S + 4)                    
⎜⎢─────────────────  ─────────────────⎥, ⎢─ - ─────────────          0        
⎜⎢      ___________        ___________⎥  ⎢2         2                         
⎜⎢S - ╲╱ S⋅(S + 4)   S + ╲╱ S⋅(S + 4) ⎥  ⎢                                    
⎜⎢                                    ⎥  ⎢                         ___________
⎜⎣        1                  1        ⎦  ⎢                   S   ╲╱ S⋅(S + 4) 
⎜                                        ⎢        0          ─ + ─────────────
⎝                                        ⎣                   2         2      

⎤⎞
⎥⎟
⎥⎟
⎥⎟
⎥⎟
⎥⎟
⎥⎟
⎥⎟
⎦⎠

In [5]:
P.inv()

⎡                    ⎛        ___________⎞ ⎛      ___________⎞   ⎛      ______
⎢                    ⎜  S   ╲╱ S⋅(S + 4) ⎟ ⎜S   ╲╱ S⋅(S + 4) ⎟   ⎜S   ╲╱ S⋅(S 
⎢      ___________   ⎜- ─ + ─────────────⎟⋅⎜─ - ─────────────⎟  -⎜─ - ────────
⎢S   ╲╱ S⋅(S + 4)    ⎝  2         2      ⎠ ⎝2         2      ⎠   ⎝2         2 
⎢─ - ───────────── - ─────────────────────────────────────────  ──────────────
⎢2         2                         ___________                      ________
⎢                                  ╲╱ S⋅(S + 4)                     ╲╱ S⋅(S + 
⎢                                                                             
⎢          ⎛        ___________⎞                                              
⎢          ⎜  S   ╲╱ S⋅(S + 4) ⎟ ⎛      ___________⎞                          
⎢          ⎜- ─ + ─────────────⎟⋅⎝S + ╲╱ S⋅(S + 4) ⎠                    ______
⎢          ⎝  2         2      ⎠                                  S + ╲╱ S⋅(S 
⎢          ─────────────────────────────────────────

Let $W=\sqrt{S(S+4)}$.
Then the general solution would be:

$$
\begin{aligned}
\begin{bmatrix}
a_n \\
b_n
\end{bmatrix}
& =
PDP^{-1}
\begin{bmatrix}
a_{n-1} \\
b_{n-1}
\end{bmatrix} \\
& =
PD^{n-1}P^{-1}
\begin{bmatrix}
a_0 \\
b_0
\end{bmatrix} \\
& =
\begin{bmatrix}
\frac{1}{2}(S-W)+\frac{1}{4W}(S-W)^2 & -\frac{S-W}{2W} \\
-\frac{S^2-W^2}{4W} & \frac{S+W}{2W}
\end{bmatrix}
\begin{bmatrix}
\frac{1}{2^{n-1}}(S-W)^{n-1} & 0 \\
0 & \frac{1}{2^{n-1}}(S+W)^{n-1}
\end{bmatrix}
\begin{bmatrix}
\frac{2}{S-W} & \frac{2}{S+W} \\
1 & 1 
\end{bmatrix}
\begin{bmatrix}
0 \\
S+1
\end{bmatrix} \\
& =
\begin{bmatrix}
2((S+\sqrt{S(S+4)})^{n-1}-(S-\sqrt{S(S+4)})^{n-1}) \\
(S+\sqrt{S(S+4)})^n-(S-\sqrt{S(S+4)})^n
\end{bmatrix}
\frac{S+1}{2^n\sqrt{S(S+4)}}
\end{aligned}
$$

In [6]:
def count_zero_block_strings(N,S):
    W = (S*(S+4))**0.5
    k = (S+1)/2**N/W
    return (
        ((S+W)**(N-1)-(S-W)**(N-1))*2 * k,
        ((S+W)**N-(S-W)**N) * k
    )

S, count_zero_block_strings(N,S)

(5, (1050.0000000000002, 6150.000000000001))

An one-block string can be presented as the combination of two sub-string start with 2 same characters and have same start characters. Let $c_L$ represent the count of L lenght on-block strings. Let $\alpha=S+W, \beta=S-W$

$$
\begin{aligned}
c_L
& =\frac{1}{S+1}\sum_{n=2}^{L-1}{a_n a_{L+1-n}} \\
& =\frac{S+1}{2^{L-1}W^2}\left[(L-2)(\alpha^{L-1}+\beta^{L-1})-\beta^{L-1}\sum_{n=2}^{L-1}{\left(\frac{\alpha}{\beta}\right)^{n-1}}-\alpha^{L-1}\sum_{n=2}^{L-1}{\left(\frac{\beta}{\alpha}\right)^{n-1}}\right] \\
& =\frac{S+1}{2^{L-1}W^2}\left[(L-2)(\alpha^{L-1}+\beta^{L-1})-2\frac{\alpha^{L-1}\beta-\beta^{L-1}\alpha}{\alpha-\beta}\right] \\
& =\frac{S+1}{2^{L-1}W^2}\left[(L-2)(\alpha^{L-1}+\beta^{L-1})+\frac{4S}{W}(\alpha^{L-2}-\beta^{L-2})\right] \\
& =\frac{S+1}{2W^2}\left[\left((L-2)\alpha+\frac{4S}{W}\right)\left(\frac{\alpha}{2}\right)^{L-2}+\left((L-2)\beta-\frac{4S}{W}\right)\left(\frac{\beta}{2}\right)^{L-2}\right]
\end{aligned}
$$


Although the result of $c_L$ is integer. However the floating operation is not acceptable especially for large numbers. Next is trying to turn it into pure integer operations. Apply [z-transform](https://en.wikipedia.org/wiki/Z-transform) then
$$
\begin{aligned}
C_L(z)
& =
\frac{S+1}{2W^2}\left[
\left((L-2)\alpha+\frac{4S}{W}\right)\frac{1}{1-\frac{\alpha}{2} z^-1}
+
\left((L-2)\beta-\frac{4S}{W}\right)\frac{1}{1-\frac{\beta}{2} z^-1}
\right] \\
& =
\frac{S+1}{2W^2}
\frac{(L-2)(\alpha+\beta)-(L-2)\alpha\beta z^{-1}+\frac{2S}{W}(\alpha-\beta)z^{-1}}{1-\frac{1}{2}(\alpha+\beta)z^{-1}+\frac{1}{4}\alpha\beta z^{-2}}  \\
& =
\frac{S+1}{S+4}
\frac{(L-2)+2(L-1)z^{-1}}{1 - S z^{-1} - S z^{-2}}
\end{aligned}
$$
Then,
$$
C_L(z)-S\left(C_L(z)+\frac{s+1}{S(S+4)}(L-1)z\right)z^{-1}-S\left(C_L(z)+\frac{S+1}{S(S+4)}(-L)z^2+\frac{S+1}{S(S+4)}(L-1)z\right)z^{-2}=0
$$
Then reverse z-transform
$$
\left\{
\begin{aligned}
c_L(n) & = S(c_L(n-1)+c_L(n-2)) \\
c_L(-2) & = \frac{S+1}{S(S+4)}(-L) \\
c_L(-1) & = \frac{S+1}{S(S+4)}(L-1) \\
c_L & = c_L(L-2)
\end{aligned}
\right.
$$
Remember that the number of different characters is represented as $S+1$

In [7]:
L = 5
def count_strings(L,S):
    if(L<3):
        return 0
    S=S-1
    k = (S+1)
    c2 = -L * (S+1)
    c1 = 2*(L-1) * (S+1) 
    while L>1:
        c0 = S * (c1+c2)
        c1,c2 = c0,c1
        L-=1
    return c1//S//(S+4)

count_strings(L,S+1)

510


Maldular 
$$
\left\{
\begin{aligned}
c_L(n) & = S(c_L(n-1)+c_L(n-2)) & \pmod{M}\\
c_L(-2) & = -L(S+1)\left(S(S+4)\right)^{-1} & \pmod{M} \\
c_L(-1) & = (L-1)(S+1)\left(S(S+4)\right)^{-1} & \pmod{M}\\
c_L & = c_L(L-2)
\end{aligned}
\right.
$$
where $\left(S(S+4)\right)^{-1}$ is [modular multiplicative inverse](https://en.wikipedia.org/wiki/Modular_multiplicative_inverse) get by [Extended Euclidean algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm). 

# Final solution
Remember that the number of characters in the abover formulars is $S+1$.

In [8]:
M = 10**9+7
SS = S + 1 # to avoid confusing, let SS represent the the number of characters
L = 5
def count_strings(L,SS):
    if(L<3):
        return 0
    S=SS-1
    _,k,_ = xgcd(S*(S+4),M)
    k = (S+1) * k % M
    c2 = -L * k % M
    c1 = 2*(L-1) * k % M
    while L>1:
        c0 = S * (c1+c2) % M
        c1,c2 = c0,c1
        L-=1
    return c1 % M
    
L, SS, count_strings(L,SS)

(5, 6, 510)

Run test cases

In [9]:
tc=[
    ((3, 7), 7),
    ((4, 2), 4),
    ((2, 17), 0),
    ((10, 11), 410199993)
]
[(countStrings(*args), ret) for args, ret in tc]

[(7, 7), (4, 4), (0, 0), (410199993, 410199993)]